In [1]:
# Final Variablen
API_KEY_SPOONACULAR = "146fe79a630b4afb8306a2985bedc64c"
API_KEY_REDDIT = "KAJ2xGzCOlH3hyTRADNdSBDuTBORvw"
API_CLIENTID_REDDIT = "tSemvcUBRFCTYQ"

USERNAME_REDDIT = "Short-Arrival7632"
PW_REDDIT = "cookies123"

API_REDIRECT_URL_REDDIT = "http://localhost:8888/notebooks/Dashboard.ipynb"

In [26]:
# KONFIGURATION

LIMIT_REDDIT_POSTS = 10
LIMIT_SPOONACULAR_RECIPES = 10

In [ ]:
# Auf Windows ausführen (einmalig)
!pip install praw
!pip install pandas

In [ ]:
# Auf Linux ausführen (einmalig)
!pip3 install praw
!pip3 install pandas

In [17]:
import praw
import pandas as pd
from datetime import datetime
import requests
import json

### Get subreddit data

In [3]:
reddit = praw.Reddit(client_id = API_CLIENTID_REDDIT,
                    client_secret = API_KEY_REDDIT,
                    username= USERNAME_REDDIT,
                    password = PW_REDDIT,
                    user_agent = "food_dashboard_v01")

sr = reddit.subreddit("food")

In [4]:
# retrieve new posts
new_posts = sr.new(limit=LIMIT_REDDIT_POSTS)

In [6]:
# clean data 
new_posts_lst = []
for post in new_posts:
    timestamp_created = post.created
    time_created = datetime.fromtimestamp(timestamp_created).strftime("%I:%M:%S")
    day_created = datetime.fromtimestamp(timestamp_created).strftime("%Y-%m-%d")
    
    # created time, created day, author, title, likes, downs, num_comments, text, thumbnail, url
    x = [time_created, day_created, post.author, post.title, post.ups, post.downs, post.num_comments, post.selftext, post.thumbnail, post.url]
    new_posts_lst.append(x)

1612656415.0
1612656235.0
1612656147.0
1612655651.0
1612655322.0
1612655241.0
1612654954.0
1612654782.0
1612654753.0


In [10]:
# create DataFrame
df = pd.DataFrame(data=new_posts_lst, columns=["created_time", "created_day", "author", "title", "likes", "downs", "num_comments", "text", "thumbnail", "url"])

In [11]:
df.describe()

,likes,downs,num_comments
count,9.000000,9.0,9.000000
mean,7.666667,0.0,0.777778
std,6.557439,0.0,1.092906
min,1.000000,0.0,0.000000
25%,3.000000,0.0,0.000000
50%,5.000000,0.0,0.000000
75%,10.000000,0.0,1.000000
max,20.000000,0.0,3.000000


### Get random recipes from spoonacular

In [27]:
req_url = "https://api.spoonacular.com/recipes/random?apiKey=" + API_KEY_SPOONACULAR + "&number=" + str(LIMIT_SPOONACULAR_RECIPES)

In [22]:
resp = requests.get(req_url)
jresp = json.loads(resp.text)

In [31]:
for recipe in jresp["recipes"]:
    print(recipe["aggregateLikes"])
    print(recipe["spoonacularSourceUrl"])

22
https://spoonacular.com/no-bake-fudge-brandy-brownies-653192
159
https://spoonacular.com/chicken-fajita-stuffed-bell-pepper-795751
8
https://spoonacular.com/crispy-rhubarb-pockets-640832
7
https://spoonacular.com/braised-pork-filipino-humba-635856
291
https://spoonacular.com/ultimate-chocolate-chunk-cookies-715483
26
https://spoonacular.com/rice-and-peas-with-coconut-curry-mackerel-716364
22
https://spoonacular.com/easy-to-make-spring-rolls-642129
163
https://spoonacular.com/what-to-make-for-dinner-tonight-bruschetta-style-pork-pasta-715538
11
https://spoonacular.com/refreshing-strawberry-limeade-658152
8
https://spoonacular.com/north-carolina-style-bbq-pulled-pork-653308
